In [3]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import h5py
import tensorflow as tf

In [4]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
keras.backend.image_data_format()

'channels_first'

In [6]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [7]:
x_train = x_train / 255.
x_test = x_test / 255.

In [8]:
x_train.shape

(60000, 28, 28)

In [9]:
x_train = x_train.reshape(-1, 1, 28, 28)
x_test = x_test.reshape(-1, 1, 28, 28)

In [10]:
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Sequential

In [11]:
import keras.backend.tensorflow_backend as tfback

In [12]:
def _get_available_gpus():  

    if tfback._LOCAL_DEVICES is None:  
        devices = tf.config.list_logical_devices()  
        tfback._LOCAL_DEVICES = [x.name for x in devices]  
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]


tfback._get_available_gpus = _get_available_gpus

In [13]:
lenet = Sequential()
lenet.add(Conv2D(6, kernel_size=3, strides=1, padding='same', input_shape=(1, 28, 28), activation="relu", \
                name='conv1', data_format="channels_first"))

lenet.add(MaxPool2D(pool_size=2, strides=2, name='pool1', data_format='channels_first'))

lenet.add(Conv2D(16, kernel_size=5, strides=1, padding='valid', activation="relu", \
                 name='conv2', data_format="channels_first"))

lenet.add(MaxPool2D(pool_size=2, strides=2, name='pool2', data_format="channels_first"))
lenet.add(Flatten(name='flatten', data_format="channels_last"))
lenet.add(Dense(120, activation="relu", name='fc1'))
lenet.add(Dense(84, activation="relu", name='fc2'))
lenet.add(Dense(10, activation='softmax', name='softmax'))

In [14]:
lenet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 6, 28, 28)         60        
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 6, 14, 14)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 10, 10)        2416      
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 16, 5, 5)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 120)               48120     
_________________________________________________________________
fc2 (Dense)                  (None, 84)               

In [22]:
lenet.compile('sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# lenet.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=[x_test, y_test])

In [17]:
# lenet.save('LeNet_NCHW.h5')

In [28]:
lenet.load_weights("LeNet_NCHW2.h5")

In [29]:
image_7 = np.expand_dims(x_test[0], axis=0)
predictions = lenet.predict(image_7)

In [30]:
predictions

array([[4.1750793e-13, 1.0217871e-07, 4.7025295e-09, 7.9027460e-09,
        4.6409793e-11, 4.0204432e-14, 3.6066829e-20, 9.9999988e-01,
        5.8345616e-12, 4.1460571e-10]], dtype=float32)

In [31]:
lenet.evaluate(x_test,y_test)

10000/10000 [==============================] - 1s 107us/step


[0.05243030259209918, 0.9857000112533569]

In [42]:
from keras.models import Model

In [53]:
#### CONV 1 #####

In [43]:
f = h5py.File("LeNet_NCHW2.h5","r")
f['model_weights'].keys()

<KeysViewHDF5 ['conv1', 'conv2', 'fc1', 'fc2', 'flatten', 'pool1', 'pool2', 'softmax']>

In [44]:
conv1_kernel=f['model_weights']['conv1']['conv1']['kernel:0'][:]

In [45]:
conv1_kernel.shape

(3, 3, 1, 6)

In [46]:
conv1_kernel= np.moveaxis(conv1_kernel,-1,0)
conv1_kernel= np.moveaxis(conv1_kernel,-1,1)
conv1_kernel.shape

(6, 1, 3, 3)

In [48]:
ofilter=6
ifilter=1
ksize=3
with open("C1_weights.h","w") as f:
    ofm=0
    for i in conv1_kernel:
        ofm+=1
        f.write("\t{\n")
        f.write("\t\t{\n")
        kernelh=0
        for j in i[0]:
            kernelh+=1
            f.write("\t\t\t{")
            kernel=0
            for k in j:
                kernel+=1
                f.write(str(k))
                if(kernel!=ksize):
                    f.write(",")
            f.write("}")
            if(kernelh!=ksize):
                f.write(",\n")
        f.write("\n\t\t}")
        f.write("\n\t}")
        if(ofm!=ofilter):
            f.write(",\n")

In [49]:
f = h5py.File("LeNet_NCHW2.h5","r")

In [50]:
conv1_bias = f['model_weights']['conv1']['conv1']['bias:0'][:]

In [51]:
conv1_bias.shape

(6,)

In [52]:
with open("C1_bias.h","w") as f:
    b=0
    f.write("\t{")
    for i in conv1_bias:
        b+=1
        f.write(str(i))
        if(b!=6):
            f.write(", ")
    f.write("};")

In [54]:
#### CONV 2 #####

In [65]:
f = h5py.File("LeNet_NCHW2.h5","r")
conv2_kernel=f['model_weights']['conv2']['conv2']['kernel:0'][:]
conv2_kernel= np.moveaxis(conv2_kernel,-1,0)
conv2_kernel= np.moveaxis(conv2_kernel,-1,1)
conv2_bias=f['model_weights']['conv2']['conv2']['bias:0'][:]
print(conv2_kernel.shape)
print(conv2_bias.shape)

(16, 6, 5, 5)
(16,)


In [72]:
ofilter=16
ifilter=6
ksize=5
with open("C2_weights.h","w") as f:
    ofm=0
    
    for i in conv2_kernel:
        ofm+=1
        f.write("\t{\n")
        ifm=0
        for l in i:
            ifm+=1
            f.write("\t\t{\n")
            kernelh=0
            for j in l:
                kernelh+=1
                f.write("\t\t\t{")
                kernel=0
                for k in j:
                    kernel+=1
                    f.write(str(k))
                    if(kernel!=ksize):
                        f.write(",")
                f.write("}")
                if(kernelh!=ksize):
                    f.write(",\n")
                    
            f.write("\n\t\t}")
            if(ifm!=ifilter):
                f.write(",\n")
        f.write("\n\t}")
        if(ofm!=ofilter):
            f.write(",\n")

In [74]:
with open("C2_bias.h","w") as f:
    b=0
    f.write("\t{")
    for i in conv2_bias:
        b+=1
        f.write(str(i))
        if(b!=16):
            f.write(", ")
    f.write("};")

In [78]:
########   FC 1   ##########
f = h5py.File("LeNet_NCHW2.h5","r")
fc1_kernel=f['model_weights']['fc1']['fc1']['kernel:0'][:]
fc1_kernel= np.moveaxis(fc1_kernel,-1,0)
fc1_bias=f['model_weights']['fc1']['fc1']['bias:0'][:]
print(fc1_kernel.shape)
print(fc1_bias.shape)

(120, 400)
(120,)


In [85]:
ofilter=120
ifilter=400

with open("FC1_weights.h","w") as f:
    ofm=0
    f.write("\t{\n")
    for i in fc1_kernel:
        ofm+=1
        ifm=0
        f.write("\t\t{")
        for l in i:
            ifm+=1
            f.write(str(l))
            if(ifm!=ifilter):
                f.write(",")
        f.write("}")
        if(ofm!=ofilter):
            f.write(",\n")
    f.write("\n\t};")

In [84]:
ofilter=120
with open("FC1_bias.h","w") as f:
    b=0
    f.write("\t{")
    for i in fc1_bias:
        b+=1
        f.write(str(i))
        if(b!=ofilter):
            f.write(", ")
    f.write("};")

In [86]:
########   FC 2   ##########
f = h5py.File("LeNet_NCHW2.h5","r")
fc2_kernel=f['model_weights']['fc2']['fc2']['kernel:0'][:]
fc2_kernel= np.moveaxis(fc2_kernel,-1,0)
fc2_bias=f['model_weights']['fc2']['fc2']['bias:0'][:]
print(fc2_kernel.shape)
print(fc2_bias.shape)

(84, 120)
(84,)


In [89]:
ofilter=84
ifilter=120

with open("FC2_weights.h","w") as f:
    ofm=0
    f.write("\t{\n")
    for i in fc2_kernel:
        ofm+=1
        ifm=0
        f.write("\t\t{")
        for l in i:
            ifm+=1
            f.write(str(l))
            if(ifm!=ifilter):
                f.write(",")
        f.write("}")
        if(ofm!=ofilter):
            f.write(",\n")
    f.write("\n\t};")

In [90]:
ofilter=84
with open("FC2_bias.h","w") as f:
    b=0
    f.write("\t{")
    for i in fc2_bias:
        b+=1
        f.write(str(i))
        if(b!=ofilter):
            f.write(", ")
    f.write("};")

In [91]:
########   FC 3   ##########
f = h5py.File("LeNet_NCHW2.h5","r")
soft_kernel=f['model_weights']['softmax']['softmax']['kernel:0'][:]
soft_kernel= np.moveaxis(soft_kernel,-1,0)
soft_bias=f['model_weights']['softmax']['softmax']['bias:0'][:]
print(soft_kernel.shape)
print(soft_bias.shape)

(10, 84)
(10,)


In [92]:
ofilter=10
ifilter=84

with open("FC3_weights.h","w") as f:
    ofm=0
    f.write("\t{\n")
    for i in soft_kernel:
        ofm+=1
        ifm=0
        f.write("\t\t{")
        for l in i:
            ifm+=1
            f.write(str(l))
            if(ifm!=ifilter):
                f.write(",")
        f.write("}")
        if(ofm!=ofilter):
            f.write(",\n")
    f.write("\n\t};")

In [93]:
ofilter=10
with open("FC3_bias.h","w") as f:
    b=0
    f.write("\t{")
    for i in soft_bias:
        b+=1
        f.write(str(i))
        if(b!=ofilter):
            f.write(", ")
    f.write("};")